Implement a first-order (Frank-Wolfe) algorithm for the optimal execution problem, using Gurobi as the LP solver.  Feel free to reuse the code that I have uploaded.  Compare with they Python code that you wrote.
As before, your code should take as input the values $\alpha$, $\pi$, $N$ (number of shares to sell) and $T$ (number of periods available).  It should output the optimal execution in each period.


In [38]:
import sys
import math
#from gurobipy import *
import numpy as np
from sympy import symbols, diff
from scipy.optimize import check_grad

def mif(x,alpha,pi):
    mi = (1-alpha*x**pi)
    return mi

def myfunction(x):
    mi=mif(x,alpha,pi)
    multi = np.cumprod(mi)
    revenue = np.sum(x*multi)
    return revenue

This function return the objective. For a vector $x$ where $x[i]$ represents the number of shares sell at time i, the objective is given by:
$$obj(x) = \sum_{i=1}^T x_i\Pi_{j=1}^i f(x_j)$$

In [39]:
#This grad_sub fucntion computes the gradient of the objective function at a vector x.
def grad_sub(x):
    gradient = np.zeros(len(x))
    fx =mif(np.array(x),alpha,pi)
    cum_f = np.cumprod(fx)

    fx_decal = np.array([1]+list(fx[:(len(x)-1)]))
    
    fdec_cum = np.cumprod(fx_decal)
    
    matrix_f =  np.triu(np.array([fx,]*len(x))) + np.tril(np.ones((len(x),len(x))),-1)
    
    matrix_cum_f = np.cumprod(matrix_f,axis=1)
    
    for i in range(len(x)-1):
        if (x[i] == 0):
            #When x[i] = 0, gradient[i] is not define so we set it arbitrarly to a big number.
            gradient[i] = -100000
        else:
            gradient[i] = fdec_cum[i]*(1 - alpha*(pi+1)*(x[i]**pi) - alpha*pi*(x[i]**(pi-1))*np.sum(x[(i+1):]*matrix_cum_f[(i+1),(i+1):]))
    gradient[len(x)-1] = np.prod(fx[:(len(x)-1)])*(1 - alpha*(pi+1)*(x[len(x)-1]**(pi)))
    
    return gradient

def gradcomp1(n, x):  
    gradient = grad_sub(x)
    return gradient

Step 2: Optimal execution algorithm
In this algorithm, we find the optimal execution with a first-order method. That is to say, we iterate on the following:
 1. Compute gradient at current point
 2. Compute slacks at current point
 3. Set up step computation LP
 4. Solve LP
 5. Get solution
 6. Step-size computation
 7. Move to new point
We can compute the gradient and the slacks at the current point with the functions we previously define.
With the function steplp that uses Gurobi, we set up and solve the LP to move to a new point that increases the obj. 
Finally, we use the function loop to perform those operations several times.

The function steplp has the follwing inputs:

t, which represents the numer of dates at which we can trade shares.
x, which is our current trading schedule i.e. x[i] is how many shares we are selling at time i.
slacks, which is a vector which size is the number of constraints that are inequalities. 
In out problem slak is equal to x because we only have the constraints 𝑥𝑖≥0
gradient which is the gradient of the objective function at point x

In [40]:
def compute_slacks(x,N):
    lslacks = -x
    uslacks = N-x
    return lslacks, uslacks

def steplp(T, x):
    m = Model("feasible")
    m.ModelSense = -1 #maximize
    
    gradient = gradcomp1 (T,x)
    #create variables and put into a dictionary
    deltavar = {}
    for j in range(T):
        deltavar[j] = m.addVar(lb = -1.0, ub = 1.0, obj = gradient[j], vtype = GRB.CONTINUOUS, name = "delta_"+str(j+1))

    # Update model to integrate new variables
    m.update()
    #add first constraint
    xpr = LinExpr()
    for i in range(T):
        xpr += deltavar[i]
    m.addConstr(xpr == 0, name="const1")
    
    #add constraint
    lslacks, uslacks = compute_slacks(x,N)
    for j in range(T):
        m.addConstr(deltavar[j]-lslacks[j] >= 0 , name="constl"+str(j+1))
        m.addConstr(deltavar[j]-uslacks[j] <= 0 , name="constu"+str(j+1))
    
    m.update()
    m.optimize()
    
    code = "optimal"

    if m.status == GRB.status.INF_OR_UNBD:
        print('->LP infeasible or unbounded\n')
        code = "inf_or_unbd"
    if m.status == GRB.status.UNBOUNDED:
        print('->LP unbounded\n')
        code = "unbd"
    if m.status == GRB.status.INFEASIBLE:
        print('->LP infeasible\n')
        code = "infeasible"


    deltasol = np.zeros(T)
    for j in range(T):
        deltasol[j] = deltavar[j].x

    return deltasol, code=="optimal"

In [41]:
#2.f Step-size computation
def linesearch(x,deltasol,Size=100):
    #h/Size is stepsize
    h=np.arange(Size+1)
    val=np.zeros(Size+1)
    for i in h:
        val[i] = myfunction(x+(h[i]/Size)*deltasol)
    print(val)
    h_opt=np.argmax(val)/Size
    return h_opt

#2. Iterate
def loop(T,x,deltasol,deltabool,h_opt,stop_dif=0.001):
    
    if deltabool==True:
        #2.g Move to new point
        x_1=x+h_opt*deltasol
        if np.nonzero(deltasol)==True:
            val=myfunction(x)
            return x,val
        elif myfunction(x_1)<myfunction(x):
            print('the objective value decreases after the movement')
            val=myfunction(x)
            return x,val
        elif myfunction(x_1)-myfunction(x)<stop_dif:
            val=myfunction(x_1)
            return x_1,val
        else:
            x=x_1
#             print(x)
            deltasol,deltabool=steplp(T, x)
            h_opt=linesearch(x,deltasol,Size=100)
            return loop(T,x,deltasol,deltabool,h_opt,stop_dif=0.001)
    else:
        print('cannot find a direction of movement')
        return None

In [42]:
def run(a,b,c,d):
    global alpha,pi,N,T
    alpha,pi,N,T = a,b,c,d
    
    gradient = np.empty(T)
    print(gradient)
    x = np.repeat(N/T,T)
    gradient =  gradcomp1(T, x)
    print(gradient)
    deltasol, deltabool = steplp(T, x)
    print(deltasol)
    h_opt = linesearch(x,deltasol)
    print(h_opt)
    x,val = loop(T,x,deltasol,deltabool,h_opt,stop_dif=0.001)
    
    return x, val


In [43]:
run(0.001,0.1,1000,20)

[0.99560491 0.99427621 0.99294948 0.99162471 0.99030191 0.98898105
 0.98766215 0.9863452  0.9850302  0.98371714 0.98240603 0.98109685
 0.97978961 0.9784843  0.97718092 0.97587947 0.97457995 0.97328234
 0.97198666 0.97069289]
[0.99560491 0.99427621 0.99294948 0.99162471 0.99030191 0.98898105
 0.98766215 0.9863452  0.9850302  0.98371714 0.98240603 0.98109685
 0.97978961 0.9784843  0.97718092 0.97587947 0.97457995 0.97328234
 0.97198666 0.97069289]


NameError: name 'Model' is not defined